In [1]:
 # ✅ Define your branch name as a string
BRANCH = "yunxiu-branch"

# ✅ Clone your GitHub repo and switch to your branch
!git clone -b $BRANCH https://github.com/xiuxiuface/dsa4213-project.git

# ✅ Move into the project folder
%cd dsa4213-project


Cloning into 'dsa4213-project'...
remote: Enumerating objects: 169, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 169 (delta 49), reused 36 (delta 24), pack-reused 88 (from 2)
Receiving objects: 100% (169/169), 78.85 MiB | 41.99 MiB/s, done.
Resolving deltas: 100% (62/62), done.
/kaggle/working/dsa4213-project


In [2]:
!pip install rank-bm25 sentence-transformers faiss-cpu transformers torch rouge-score bert-score textstat pandas numpy tqdm openpyxl

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 58.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 86.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━

In [11]:
"""
Test Subset Code - Quick Testing with Small Sample
使用小样本快速测试代码是否正常运行
"""
from RAG_Summarization_Complete import (
    BM25Retriever,
    FAISSRetriever,
    HybridRetriever,
    SummarizationModel,
    RAGPipeline,
    RAGEvaluator,
    load_data,
    prepare_corpus
)
import pandas as pd
import numpy as np
from tqdm import tqdm
import json
import os
from typing import List, Dict, Tuple

# Retrieval libraries
from rank_bm25 import BM25Okapi
from sentence_transformers import SentenceTransformer
import faiss

# Generation
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Evaluation
from rouge_score import rouge_scorer
from bert_score import score as bert_score
import textstat
# ============================================================================
# QUICK TEST WITH SUBSET
# ============================================================================

def quick_test(sample_size: int = 50):
    """
    Quick test with small sample to verify everything works
    
    Args:
        sample_size: Number of samples to test (default: 50)
    """
    print("="*80)
    print(f"QUICK TEST MODE - Using {sample_size} samples")
    print("="*80)
    
    # 1. Load data
    print("\n[1/5] Loading data...")
    df = load_data("rag_dataset (3).xlsx")
    
    # 2. Sample subset
    print(f"\n[2/5] Sampling {sample_size} questions...")
    test_df = df.drop_duplicates(subset=["question_id"]).sample(
        n=sample_size, 
        random_state=42
    ).reset_index(drop=True)
    print(f"✓ Test set size: {len(test_df)}")
    
    # 3. Prepare corpus
    print("\n[3/5] Preparing corpus...")
    corpus, passage_ids = prepare_corpus(df)
    
    # 4. Initialize models
    print("\n[4/5] Initializing models...")
    
    # Retrievers
    print("  - BM25...")
    bm25 = BM25Retriever(corpus)
    
    print("  - FAISS...")
    faiss = FAISSRetriever(corpus)
    
    # Summarizer
    print("  - FLAN-T5...")
    summarizer = SummarizationModel("google/flan-t5-base")
    
    # 5. Test each retriever
    print("\n[5/5] Testing retrievers...")
    
    configs = [
        {"name": "BM25_k3", "retriever": bm25, "k": 3},
        {"name": "FAISS_k3", "retriever": faiss, "k": 3},
    ]
    
    evaluator = RAGEvaluator()
    results = []
    
    for config in configs:
        print(f"\n  Testing {config['name']}...")
        
        pipeline = RAGPipeline(
            retriever=config['retriever'],
            summarizer=summarizer,
            k=config['k']
        )
        
        predictions = []
        references = []
        
        for _, row in tqdm(test_df.iterrows(), total=len(test_df), desc=f"  {config['name']}"):
            pred = pipeline.run(row['question'])
            predictions.append(pred)
            references.append(row['answer'])
        
        metrics, fk_grades = evaluator.evaluate_batch(predictions, references)
        
        result = {
            "config": config['name'],
            **metrics
        }
        results.append(result)
        
        print(f"    ROUGE-1: {metrics['ROUGE-1']:.4f}")
        print(f"    ROUGE-2: {metrics['ROUGE-2']:.4f}")
        print(f"    BERTScore: {metrics['BERTScore_F1']:.4f}")
        print(f"    Avg FK: {metrics['Avg_FK_Grade']:.2f}")
    
    # Save test results
    os.makedirs("test_results", exist_ok=True)
    
    results_df = pd.DataFrame(results)
    # results_df.to_csv("quick_test_results.csv", index=False)
    results_dict = results_df.to_dict(orient='records')
    with open("quick_test_metrics.json", "w") as f:
        json.dump(results_dict, f, indent=2)
    
    # test_df['rag_summary_bm25'] = [pipeline.run(q) for q in test_df['question'][:10]]
    test_df.to_csv("test_sample_outputs.csv", index=False)
    
    print("\n" + "="*80)
    print("✓ QUICK TEST COMPLETED")
    print("="*80)
    print("\nResults:")
    print(results_df.to_string())
    print(f"\nSaved to: test_results/")
    
    return results_df

# ============================================================================
# INSPECT SPECIFIC EXAMPLES
# ============================================================================

def inspect_examples(n: int = 3):
    """
    Inspect a few examples in detail
    
    Args:
        n: Number of examples to inspect
    """
    print("\n" + "="*80)
    print(f"INSPECTING {n} EXAMPLES")
    print("="*80)
    
    # Load data
    df = load_data("rag_dataset (3).xlsx")
    corpus, _ = prepare_corpus(df)
    
    # Sample
    sample = df.drop_duplicates(subset=["question_id"]).sample(n=n, random_state=42)
    
    # Initialize
    bm25 = BM25Retriever(corpus)
    faiss = FAISSRetriever(corpus)
    summarizer = SummarizationModel("google/flan-t5-base")
    
    for i, (_, row) in enumerate(sample.iterrows(), 1):
        print(f"\n{'='*80}")
        print(f"EXAMPLE {i}/{n}")
        print(f"{'='*80}")
        
        print(f"\n📝 Question:\n{row['question']}")
        print(f"\n✅ True Answer:\n{row['answer']}")
        
        # BM25
        print(f"\n🔍 BM25 Retrieved Context (top-3):")
        bm25_ctx = bm25.retrieve(row['question'], k=3)
        for j, doc in enumerate(bm25_ctx, 1):
            print(f"  [{j}] {doc[:200]}...")
        
        bm25_summary = summarizer.generate(row['question'], bm25_ctx)
        print(f"\n🤖 BM25 RAG Summary:\n{bm25_summary}")
        print(f"   FK Grade: {textstat.flesch_kincaid_grade(bm25_summary):.2f}")
        
        # FAISS
        print(f"\n🔍 FAISS Retrieved Context (top-3):")
        faiss_ctx = faiss.retrieve(row['question'], k=3)
        for j, doc in enumerate(faiss_ctx, 1):
            print(f"  [{j}] {doc[:200]}...")
        
        faiss_summary = summarizer.generate(row['question'], faiss_ctx)
        print(f"\n🤖 FAISS RAG Summary:\n{faiss_summary}")
        print(f"   FK Grade: {textstat.flesch_kincaid_grade(faiss_summary):.2f}")

# ============================================================================
# MAIN TEST EXECUTION
# ============================================================================

if __name__ == "__main__":
    import sys
    
    if len(sys.argv) > 1 and sys.argv[1] == "inspect":
        # Inspect mode: python test_subset_code.py inspect
        inspect_examples(n=5)
    else:
        # Quick test mode: python test_subset_code.py
        quick_test(sample_size=50)
    
    print("\n✨ Done!")


QUICK TEST MODE - Using 50 samples

[1/5] Loading data...
Loaded dataset: (46610, 6)
Columns: ['question_id', 'question', 'answer', 'final_decision', 'passage', 'passage_id']

[2/5] Sampling 50 questions...
✓ Test set size: 50

[3/5] Preparing corpus...
Unique passages: 46609

[4/5] Initializing models...
  - BM25...
Initializing BM25...
✓ BM25 ready
  - FAISS...
Initializing FAISS with all-MiniLM-L6-v2...
Creating passage embeddings...


Batches:   0%|          | 0/1457 [00:00<?, ?it/s]

✓ FAISS index ready with 46609 vectors
  - FLAN-T5...
Loading summarization model: google/flan-t5-base...
✓ Model loaded on cuda

[5/5] Testing retrievers...

  Testing BM25_k3...


  BM25_k3: 100%|██████████| 50/50 [00:46<00:00,  1.09it/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


    ROUGE-1: 0.3199
    ROUGE-2: 0.1410
    BERTScore: 0.8788
    Avg FK: 16.65

  Testing FAISS_k3...


  FAISS_k3: 100%|██████████| 50/50 [00:35<00:00,  1.43it/s]
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


    ROUGE-1: 0.3161
    ROUGE-2: 0.1397
    BERTScore: 0.8772
    Avg FK: 16.33

✓ QUICK TEST COMPLETED

Results:
     config   ROUGE-1   ROUGE-2   ROUGE-L  BERTScore_F1  Avg_FK_Grade
0   BM25_k3  0.319931  0.141014  0.255285      0.878768     16.654886
1  FAISS_k3  0.316119  0.139742  0.262601      0.877222     16.332556

Saved to: test_results/

✨ Done!
